## Experiment logger
The logger stores experimental data in a single SQLite database. It is intended to be fast and lightweight, but record all necessary meta data and timestamps for experimental trials.

Most of the entries are stored as JSON strings in the database tables; any object that can be serialised by Python's `json` module can be added directly.

### Connecting to the database

In [1]:
from experimentlog import ExperimentLog

## open a connection to a database; will be created if it does not exist.
# here we use a memory database so the results are not stored to disk
e = ExperimentLog(":memory:")

### Setting up the database
When a log is set up for the first time, the database needs to be configured for the experimental sessions. 

Each sensor/information **stream** needs to be registered with the database. This could be individual sensors like a mouse (x,y) time series, or questionnaire results.
Every entry in the log stream maintains the time, tag and a "valid" flag for each entry, plus the JSON representing the data logged.

In [2]:
# check if we've already set everything up
if e.get_stage()=="init":
    
    # we'll log from two streams: the mouse time series, and a post-condition questionnaire
    e.register_stream(name="mouse", description="A time series of x,y cursor positions",
                   # the data is optional, and can contain anything you want 
                  data={
                    "sample_rate": 60,
                    "dpi": 3000,
                    "mouse_device":"Logitech MX600"})

    e.register_stream(name="satisfaction", 
                   description="A simple satisfaction score",
                   # here, we store the questions used for future reference
                  data={
                    "questions":["How satisfied were you with your performance?",
                                "How satisfied were you with the interface?"]}
                    )


## Sessions
**ExperimentLog** uses the concept of *sessions* to manage experimental data. Sessions are much like folders in a filesystem and usually form a hierarchy, for example:
    
    /
        Experiment1/
            ConditionA/
                Rep1/
                Rep2/
                Rep3/
            ConditionB/
                Rep1/
                Rep2/
                Rep3/
                
        Experiment 2
            ConditionA/
                Rep1/
                Rep2/
                Rep3/
                Rep4/
            ConditionC/
                Rep1/
                Rep2/
                Rep3/
                Rep4/
    

Each *session* type has **metadata** attached to it; for example giving the parameters for a given condition. 

When an experiment is run, **instances** of sessions are created, like files inside the filesystem.

In [3]:
if e.get_stage()=="init":
    # We'll register an experiment, with three different conditions
    e.register_session("Experiment", description="The main experiment", 
                           data={"target_size":40.0, "cursor_size":5.0})
    e.register_session("ConditionA",description="Condition A:circular targets", 
                           data={"targets":["circle"]})
    e.register_session("ConditionB", description="Condition B:square targets", 
                           data={"targets":["square"]})
    e.register_session("ConditionC", description="Condition C:mixed targets", 
                           data={"targets":["circle","mixed"]})


We'd usually only want to do this once-ever; this setup procedure can be recorded by changing the database **stage**:

In [39]:
# mark the database as ready to log data
e.set_stage("setup")

## Users
Each instance of a session (usually) involves experimental subjects. Each user should be registered, and then attached to a recording session. Multiple users can be attached to one session (e.g. for experiments with groups) but normally there will just be one user.

The `pseudo` module can generate pronounceable, random, verifiable pseudonyms for subjects.


In [5]:
import pseudo
user = pseudo.get_pseudo()
print user

SIKOW-ESAWI


In [6]:
# now register the user with the database
e.register_user(name=user, user_vars={"age":30, "leftright":"right"})


'SIKOW-ESAWI'

## Runs
Finally, each **run** of the experimental session is logged. If there are any variables that change on a per-run basis (e.g. calibration parameters) they can be stored here.

The experimenter running this trial should be specified for each run.

In [25]:
e.start_run(experimenter="JHW")

In [26]:
# attach the user to this experimental run
e.add_active_user(user)
# enter conditionA of experiment
e.enter_session("Experiment")
e.enter_session("ConditionA")

In [27]:
e.log("mouse", data={"x":0, "y":10})

5

In [28]:
e.log("mouse", data={"x":0, "y":20})

6

In [29]:
e.log("mouse", data={"x":20, "y":20})

7

In [30]:
e.log("satisfaction", data={"q1":4,"q2":5})

8

In [31]:
# move out of condition A
e.leave_session()

In [32]:
e.enter_session("ConditionB")

In [33]:
# could log more stuff...

In [34]:
e.leave_session()
e.leave_session()
# back to the root

In [35]:
e.end_run() # end the run

In [36]:
# print some results with raw SQL queries
mouse_log = e.cursor.execute("SELECT time, json FROM mouse", ())
print "\n".join([str(m) for m in mouse_log.fetchall()])

(1454596050.824, u'{"y": 10, "x": 0}')
(1454596050.838, u'{"y": 20, "x": 0}')
(1454596050.859, u'{"y": 20, "x": 20}')
(1454596134.062, u'{"y": 10, "x": 0}')
(1454596134.207, u'{"y": 20, "x": 0}')
(1454596134.35, u'{"y": 20, "x": 20}')


In [37]:
import report

In [41]:
print report.string_report(e.cursor)

# Report generated for none

----------------------------------------
#### Report date: Thu Feb 04 14:49:15 2016

----------------------------------------

## Runs
* Number of runs: 2
* Total duration recorded: 1.9 seconds
* Dirty exits: 0

----------------------------------------

## Sessions

#### /Experiment
* Runs: 2
* Duration recorded: 1.30399990082 seconds

#### /Experiment/ConditionA
* Runs: 2
* Duration recorded: 0.824000120163 seconds

#### /Experiment/ConditionB
* Runs: 2
* Duration recorded: 0.316999912262 seconds

----------------------------------------

## Users
* Unique users: 1


#### SIKOW-ESAWI
**JSON** 
 
        {
            "age": 30,
            "leftright": "right"
        }
        
Duration recorded: 0.282999992371 seconds
Paths recorded:
	/Experiment
	/Experiment/ConditionA
	/Experiment/ConditionB
----------------------------------------

## Log
* Log streams recorded: mouse,satisfaction

#### mouse
##### A time series of x,y cursor positions
**JSON** 
 
   

## SQL format
There are a few basic tables in the ExperimentLog:

#### Metadata
    meta: 
        id, Unique ID
        mtype,    Type of this metadata: one of LOG, SESSION, USER, PATH
        name,     Name of the object, e.g. user pseudonyn
        type,     (Optional) type tag
        description, (Optional) text description
        json       (Optional) JSON string holding any other metadata.

The metadata for a log, session or user, path. `mtype` specifies the kind of metadata it is. There are convenience views of this table:

    log_stream
    users
    session_meta
    paths

All have the same fields as above.

#### Session

        session: 
            id,          Unique ID
            start_time,  Time this session was started
            end_time,    Time this session was completed (if it was)
            last_time,   Last time a log was written for this session
            test_run,    If this is a test run or not
            random_seed, Random seed used for this session can be stored here
            valid,       If this session was marked valid or not
            complete,    If this session was marked completed or not
            parent,      ID of the session this session is a subsession of
            path,        ID of the full path this session belongs to
            json,        Any additional metadata

        
       run_session: (maps sessions to runs)
           id,           Unique ID
           run,          ID of the run
           session,      ID of the session
           
       user_session: (maps users to sessions)
           id,           Unique ID
           session,      ID of the session
           user,         ID of the user
           role,         (optional) String giving the role the user plays
           json,         (optional) Any session-specific user variables.
           
#### Logs

    log:
        id,         Unique ID
        time,       Timestamp
        valid,      Valid flag for this data (e.g. to mark faulty sensor data)
        stream,     ID of the stream this log belongs to
        session,    ID of the session this log entry belongs to
        json,       The log entry itself
        tag,        (optional) tag for this log entry
        
        
When a new log is created, it also creates a new view with the given stream name; so e.register_log("mouse") creates a new view table "mouse" with the same fields as `log`.

### Misc

    children: (maps sessions to all of their children sessions)
        id,     Unique ID
        parent, ID of the parent session
        child,  ID of the child session

This is automatically filled in by `enter_session()`/`leave_session()`
       
    setup:
        id,      Unique ID
        time,    Time of the stage change
        stage,   Stage of the database

This is normally only used to record whether the database is fully setup or not.

